# Filter dataset

In [1]:
2 + 2

4

In [2]:
import os

In [3]:
!nvidia-smi

Thu Dec 14 00:06:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:00:05.0 Off |                  N/A |
|  0%   28C    P8    26W / 350W |      8MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:00:06.0 Off |                  N/A |
|  0%   

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [8]:
import pyspark
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PreprocessText").getOrCreate()

In [9]:
import pyspark.sql.functions as f

def pre_process(df):
    """
    Pre-processes the DataFrame by:
    - Removing links
    - Removing emojis
    - Removing texts less than 10 words
    """
    df = df.select("Content").withColumnRenamed("Content", "text")
    # Remove links
    df = df.withColumn("text", f.regexp_replace(f.col("text"), r"https?://\S+", ""))
    # Remove emojis
    df = df.withColumn("text", f.regexp_replace(f.col("text"), r"[^\s\d\p{L}\p{Punct}]", ""))
    # Remove texts less than 10 words
    df = df.where(f.size(f.split(df.text, "\s+")) >= 50)

    return df

df = spark.read.csv("unsupervised_data.csv", header=True, multiLine=True)
df = pre_process(df)

df.show(10, truncate=60)
print(f'df.count(): {df.count()}')

+------------------------------------------------------------+
|                                                        text|
+------------------------------------------------------------+
|Я мертв, но даже в смерти зол и горд\n\nВойна добро и зло...|
|Современная гибридная война: кибер-аспект и роль кибербез...|
|Поздравляем наших побратимов из Украины с Днем защитника ...|
|В ГОСДУМЕ ОБЪЯСНИЛИ, ЗА КАКИЕ ПОСТРОЙКИ НА ДАЧАХ МОЖНО НЕ...|
|Шесть человек погибли и четверо пострадали при пожаре в н...|
|Благодаря поддержке Дорогого БРАТА-Главы ЧР, Героя России...|
|Фронтовая сводка 1 октября \n\nНа Запорожском направлении...|
|"На Настю Ивлееву подала в суд жительница Челябинской обл...|
|Завершилась очередная трудовая неделя, а значит пришло вр...|
|Это Степанакерт. Азербайджанцы свалили крест. Никакого те...|
+------------------------------------------------------------+
only showing top 10 rows



df.count(): 1347338


In [10]:
df = df.limit(50_000)

# Tokenize dataset

In [11]:
from transformers import AutoTokenizer, AutoModel
import torch

weights_path = "ai-forever/sbert_large_nlu_ru"
tokenizer = AutoTokenizer.from_pretrained(weights_path, cache_dir=".")

In [12]:
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, IntegerType, FloatType

from transformers import AutoTokenizer, AutoModel

weights_path = "ai-forever/sbert_large_nlu_ru"
tokenizer = AutoTokenizer.from_pretrained(weights_path, cache_dir=".")

def tokenize_udf(text, max_size = 512):
    encoded_inputs = tokenizer(text, return_tensors="pt")
    input_ids = encoded_inputs["input_ids"].squeeze().tolist()[:max_size]
    token_type_ids = encoded_inputs["token_type_ids"].squeeze().tolist()[:max_size]
    attention_mask = encoded_inputs["attention_mask"].squeeze().tolist()[:max_size]
    return input_ids, token_type_ids, attention_mask

tokenize_udf = udf(tokenize_udf, ArrayType(ArrayType(IntegerType())))

# Tokenize the data
tokenized_df = df.withColumn('data', tokenize_udf(df.text))
tokenized_df = tokenized_df.withColumn('input_ids', tokenized_df.data.getItem(0))
tokenized_df = tokenized_df.withColumn('token_type_ids', tokenized_df.data.getItem(1))
tokenized_df = tokenized_df.withColumn('attention_mask', tokenized_df.data.getItem(2))
tokenized_df = tokenized_df.select('input_ids', 'token_type_ids', 'attention_mask')

# Access the tokenized data
tokenized_path = 'tokenized_df.parquet'
tokenized_df.write.mode('overwrite').save(tokenized_path, format='parquet', header=True)

tokenized_df.toPandas()

,input_ids,token_type_ids,attention_mask
0,"[101, 119, 26105, 121, 750, 1180, 113, 3286, 4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[101, 28274, 85501, 844, 29241, 162, 15677, 13...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[101, 16629, 28831, 3726, 87139, 1977, 734, 18...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[101, 113, 3944, 64468, 22305, 121, 681, 2867,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[101, 4439, 1266, 5186, 107, 12648, 10226, 711...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...
49995,"[101, 16212, 102657, 28967, 12331, 660, 6049, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
49996,"[101, 5748, 133, 1049, 939, 12941, 133, 48335,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
49997,"[101, 13092, 52182, 177, 93783, 9282, 102351, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
49998,"[101, 82645, 1659, 18801, 67124, 458, 133, 194...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


# Mask pretrain

In [13]:
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM
import torch

weights_path = "ai-forever/sbert_large_nlu_ru"
tokenizer = AutoTokenizer.from_pretrained(weights_path, cache_dir=".")
model = AutoModelForMaskedLM.from_pretrained(weights_path, cache_dir=".")

Some weights of BertForMaskedLM were not initialized from the model checkpoint at ai-forever/sbert_large_nlu_ru and are newly initialized: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
tokenized_path = 'tokenized_df.parquet'
tokenized_df = spark.read.load(tokenized_path, format='parquet').toPandas()

In [15]:
from datasets import load_dataset
from datasets import Dataset

tokenized_ds = Dataset.from_pandas(tokenized_df)
tokenized_ds = tokenized_ds.train_test_split(test_size=0.1)
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5000
    })
})

In [16]:
from transformers import DataCollatorForLanguageModeling

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# if tokenizer.eos_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[EOS]'})
#     model.resize_token_embeddings(len(tokenizer))

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
from transformers import TrainingArguments, Trainer

torch.cuda.empty_cache()
batch_size = 4
training_args = TrainingArguments(
    output_dir="my_awesome_bert",
    evaluation_strategy="steps",
    save_strategy="steps", 
    eval_steps=0.05, 
    max_steps=int(400000 / batch_size), 
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    report_to="tensorboard",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=2*batch_size,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss,Validation Loss


In [1]:
2 + 2

4

- [Spark NLP Models Hub
](https://sparknlp.org/models?task=Embeddings&language=xx&edition=Spark+NLP+5.2&sort=downloads)
- [sparknlp embedings](https://sparknlp.org/api/python/reference/autosummary/sparknlp/annotator/embeddings/bert_embeddings/index.html)

In [32]:
data.text.iloc[0]

'Всё дело в буквах, отменить и мы сразу победим)))! Марков М-дак)))'

In [33]:
aug.augment(data.text.iloc[0])

['Всё дело в буквах, то и мы сразу про ) ) )! Марков ин - дак ) ) )']

In [31]:
aug.augment(data.text.iloc[-1])

['Я мертв, но даже в гибели зол и горд Война добро и зло не выбирает. Есть мысли, ручка, воинский билет. А понимаю наша рота погибает Я знала, что итогом будет смерть. Что написать врагу чтобы он понял, И смысл строк оставил на душе Солдат России это правды воин Хотя сейчас и легкая мишень. Живым меня ты точно не возьмёшь, А безоружным пред тобою не буду И если ты в окоп ко мне зайдёшь То точно позабудь о слове чудо Я буду в клочья я тебя нацист Я вечно и рядом, я как ветер Я твой последний в жизни экзорцист я СВОй читай в моем билете. Там знаю что Победа, наш трофей, Её только давно нам завещал. Строкой то говорю СВОей Чтоб враг ее пред смертью вспоминал! Я сын страны и в убеждениях твёрд У всех солдат в бою СВОя мессия Я мертв, но даже в смерти зол и горд Что я погиб за Правду и Её! Лик Д. С.']